## Installation and Imports

In [ ]:
!pwd

In [ ]:
# Configure env variables
import os
os.environ["DARM_MUJOCO_PATH"] = "/home/daniel/DARM/darm_mujoco"

In [ ]:
!python setup.py install

In [ ]:
!pip install ray[rllib] torch

In [ ]:
import ray
from ray.rllib.algorithms.sac import SACConfig
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

from ray import air, tune
from ray.air import session
from ray.air.integrations.wandb import setup_wandb
from ray.air.integrations.wandb import WandbLoggerCallback

import gym
from darm_gym_env import DARMSFEnv

## Register Environment with RLlib

In [ ]:
# env_creator = lambda env_config: gym.make("darm/DarmSFHand-v0", render_mode=None, hand_name="hand1") # DARMSFEnv(render_mode=None, reaction_time=0.08, hand_name="hand1") # 

def make_env(env_config):
    env = gym.wrappers.TimeLimit(env=DARMSFEnv(render_mode=None, reaction_time=0.08, hand_name="hand1"), max_episode_steps=100)
    return env
env_creator = lambda env_config: make_env(env_config) #gym.wrappers.TimeLimit(env=DARMSFEnv(render_mode=None, reaction_time=0.08, hand_name="hand1"), max_episode_steps=100)

register_env("darm/DarmSFHand-v0", env_creator)

## Configure and Run

In [ ]:
config = (
    SACConfig()
    .environment(
        env="darm/DarmSFHand-v0",
        normalize_actions=True
    )
    .training(
        q_model_config={
            "fcnet_activation": "relu",
            "fcnet_hiddens": [256, 256]
        },
        policy_model_config={
            "fcnet_activation": "relu",
            "fcnet_hiddens": [256, 256]
        },
        tau=0.005,
        target_entropy="auto",
        n_step=1,  # num of SGD steps per batch of data in training step
        train_batch_size=256,
        target_network_update_freq=1,
        replay_buffer_config={"type":"MultiAgentPrioritizedReplayBuffer"},
        num_steps_sampled_before_learning_starts=10_000,
        optimization_config={
          "actor_learning_rate": 0.0003,
          "critic_learning_rate": 0.0003,
          "entropy_learning_rate": 0.0003,
        },
        clip_actions=False
    )
    .rollouts(
        num_rollout_workers=3,
        rollout_fragment_length=1,
    )
    .resources(num_gpus=0)
    .evaluation(evaluation_interval=100) # For 1000 timesteps iter; 100 evals
    .reporting(
        min_sample_timesteps_per_iteration=1000,
        metrics_num_episodes_for_smoothing=5
    )
    .framework(framework="torch")
)

In [ ]:
wandb_init = dict(
    save_code=True,
    config={
        "env": "DARMSFHand-v0",
        
        "actor_learning_rate": 0.0003,
        "critic_learning_rate": 0.0003,
        "entropy_learning_rate": 0.0003,
        "framework": "torch",
        
        "num_rollout_workers": 3,
        "num_gpu": 0,
        "metrics_num_episodes_for_smoothing": 5
    },
    tags=["single_finger"],
    notes="Fixed the env to use targets that are delta increaments from the starting state. Removed velocity penalty, and used only effort penalty",
    name="Test_DARMSF_DELTA_TARGET"
    # job_type=D
    # monitor_gym=
)

In [ ]:
tuner = tune.Tuner(
    "SAC",
    run_config=air.RunConfig(
        name="Test_DARMSF_DELTA_TARGET",
        stop={"training_iteration": 10_000, "episode_reward_mean": 200},
        checkpoint_config=air.CheckpointConfig(checkpoint_at_end=True),
        callbacks=[
                WandbLoggerCallback(project="DARM", save_checkpoints=True, **wandb_init)
            ],
        local_dir="./results"
        ),
    param_space=config
)

results = tuner.fit()

2023-02-09 14:34:37,372	INFO worker.py:1538 -- Started a local Ray instance.
2023-02-09 14:34:38,879	INFO wandb.py:250 -- Already logged into W&B.


(SAC pid=27220) 2023-02-09 14:34:42,596	WARNING algorithm_config.py:488 -- Cannot create SACConfig from given `config_dict`! Property __stdout_file__ not supported.
(SAC pid=27220) 2023-02-09 14:34:42,995	INFO algorithm.py:501 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
wandb: Currently logged in as: danieladejumo. Use `wandb login --relogin` to force relogin
(raylet) [2023-02-09 14:34:47,263 E 26959 27004] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-09_14-34-35_158333_26854 is over 95% full, available space: 1199403008; capacity: 31845081088. Object creation will fail if spilling is required.
(RolloutWorker pid=27324) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=27324)   logger.warn(
(RolloutWorker pid=27324) 2023-02-09 14:34:49,388	WARNING env.py:147 -- Your env doesn't have a .spe

(RolloutWorker pid=27324) Loaded XML file successfully


(RolloutWorker pid=27325) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=27325)   logger.warn(


(RolloutWorker pid=27325) Loaded XML file successfully


(RolloutWorker pid=27327) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=27327)   logger.warn(


(RolloutWorker pid=27327) Loaded XML file successfully
(SAC pid=27220) Loaded XML file successfully


(SAC pid=27220) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(SAC pid=27220)   logger.warn(
(SAC pid=27220) 2023-02-09 14:34:50,987	WARNING env.py:147 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(SAC pid=27220) 2023-02-09 14:34:51,023	WARNING util.py:66 -- Install gputil for GPU system monitoring.
(SAC pid=27220) 2023-02-09 14:34:51,348	WARNING multi_agent_prioritized_replay_buffer.py:215 -- Adding batches with column `weights` to this buffer while providing weights as a call argument to the add method results in the column being overwritten.


(raylet) [2023-02-09 14:34:57,270 E 26959 27004] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-09_14-34-35_158333_26854 is over 95% full, available space: 1199267840; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-09 14:35:07,276 E 26959 27004] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-09_14-34-35_158333_26854 is over 95% full, available space: 1199169536; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-09 14:35:17,287 E 26959 27004] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-09_14-34-35_158333_26854 is over 95% full, available space: 1199026176; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-09 14:35:27,295 E 26959 27004] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-09_14-34-35_158333_26854 is over 95% full, available space: 1198841856; capacity: 31845081088. Object creation will fa